<a href="https://colab.research.google.com/github/Kostogot/Two/blob/main/Binan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
 import pandas as pd # for read file 
import warnings; warnings.simplefilter('ignore') #ignore warnings
pd.set_option('display.max_columns', None) # display all column
B = pd.read_excel('/binan.xlsx', 2) # read file
#B

In [22]:
P = [] # list
Psh = [] # list for short
for n in range(0,2): # cycle for profit cells 
  V = B['volume'][n] # read volume
  if B['operation'][n] == 'buy' or B['operation'][n] == 'buysh': # if operation buy
    B['ostatok'][n] = V # the remainder does not change
    if B['operation'][n] == 'buy':
      P.append([B['price'][n],n]) # price add 
    else:
      Psh.append([B['price'][n],n])
  else:
    B['profit'][n] = 0 # clear cell for calculation 
    B['profit rub'][n] = 0
    B['%'][n] = 0
    B['%R'][n] = 0
    ComS = 0 # sell commission in percent
    ComB = 0 # buy commission in percent
    ComBP = 0 # buy commission for profit
    ComBPR = 0 # buy commission for profit rub
    ComSR = 0 # sell commission in percent rub
    j = 0 
    if B['operation'][n] == 'sell':
      P.sort() # sort of price, from min to max
      for i in range(0,len(P)): # open purchase cycle
        if B['ostatok'][P[i][1]] > 0 and V > 0: # if the purchase is not exhausted and there is something to sell 
          if V > B['ostatok'][P[i][1]]: # sell more buy volume 
            B['profit'][n] += (B['price'][n]-B['price'][P[i][1]])*B['ostatok'][P[i][1]] # calculation profit
            B['profit rub'][n] += (B['price'][n]*B['$'][n]-B['price'][P[i][1]]*B['$'][P[i][1]])*B['ostatok'][P[i][1]] # calculation profit rub
            ComS += 1/(B['price'][P[i][1]]*B['ostatok'][P[i][1]]) 
            ComBP += B['fee'][P[i][1]]*B['ostatok'][P[i][1]]/B['volume'][P[i][1]] 
            V -= B['ostatok'][P[i][1]] # reducing the remaining money
            B['ostatok'][P[i][1]] = 0
          else:
            B['profit'][n] += (B['price'][n]-B['price'][P[i][1]])*V # calculation profit
            B['profit rub'][n] += (B['price'][n]*B['$'][n]-B['price'][P[i][1]]*B['$'][P[i][1]])*V # calculation profit rub
            ComS += 1/(B['price'][P[i][1]]*V)  
            ComBP += B['fee'][P[i][1]]*V/B['volume'][P[i][1]]
            B['ostatok'][P[i][1]] = round(B['ostatok'][P[i][1]]-V, 10) # calculate remainder from buy, round for show null without e+   
            V = 0
          ComB += B['fee'][P[i][1]]/(B['price'][P[i][1]]*B['volume'][P[i][1]])
          ComBPR += ComBP*B['$'][P[i][1]]
          B['%'][n] += (B['price'][n]-B['price'][P[i][1]])/B['price'][P[i][1]] # calculate base for %  
          B['%R'][n] += (B['price'][n]*B['$'][n]-B['price'][P[i][1]]*B['$'][P[i][1]])/(B['price'][P[i][1]]*B['$'][P[i][1]]) # calculate base for % R
          ComSR += ComS/B['$'][P[i][1]]
          j += 1         
    else:
      Psh.sort(reverse=True) # sort of price, from min to max
      for i in range(0,len(Psh)): # open purchase cycle        
        if B['ostatok'][Psh[i][1]] > 0 and V > 0: # if the purchase is not exhausted and there is something to sell 
          if V > B['ostatok'][Psh[i][1]]: # sell more buy volume 
            B['profit'][n] += (B['price'][Psh[i][1]]-B['price'][n])*B['ostatok'][Psh[i][1]] # calculation profit
            B['profit rub'][n] += (B['price'][Psh[i][1]]*B['$'][Psh[i][1]]-B['price'][n]*B['$'][n])*B['ostatok'][Psh[i][1]] # calculation profit rub
            ComS += 1/(B['price'][Psh[i][1]]*B['ostatok'][Psh[i][1]]) 
            ComBP += B['fee'][Psh[i][1]]*B['ostatok'][Psh[i][1]]/B['volume'][Psh[i][1]]
            V -= B['ostatok'][Psh[i][1]] # reducing the remaining money
            B['ostatok'][Psh[i][1]] = 0
          else:
            B['profit'][n] += (B['price'][Psh[i][1]]-B['price'][n])*V # calculation profit
            B['profit rub'][n] += (B['price'][Psh[i][1]]*B['$'][Psh[i][1]]-B['price'][n]*B['$'][n])*V # calculation profit rub
            ComS += 1/(B['price'][Psh[i][1]]*V)  
            ComBP += B['fee'][Psh[i][1]]*V/B['volume'][Psh[i][1]]
            B['ostatok'][Psh[i][1]] = round(B['ostatok'][Psh[i][1]]-V, 10) # calculate remainder from buy, round for show null without e+   
            V = 0
          ComB += B['fee'][Psh[i][1]]/(B['price'][Psh[i][1]]*B['volume'][Psh[i][1]])
          ComBPR += ComBP*B['$'][Psh[i][1]]
          B['%'][n] += (B['price'][Psh[i][1]]-B['price'][n])/B['price'][Psh[i][1]] # calculate base for %
          B['%R'][n] += (B['price'][Psh[i][1]]*B['$'][Psh[i][1]]-B['price'][n]*B['$'][n])/(B['price'][Psh[i][1]]*B['$'][Psh[i][1]]) # calculate base for % rub
          ComSR += ComS/B['$'][Psh[i][1]]
          j += 1     
    B['profit'][n] = B['profit'][n]-B['fee'][n]-ComBP
    B['profit rub'][n] = B['profit rub'][n]-B['fee'][n]*B['$'][n]-ComBPR
    B['%'][n] = (B['%'][n]-B['fee'][n]/j*ComS-ComB)/j*100  # calculate % of profit 
    B['%R'][n] = (B['%R'][n]-B['fee'][n]*B['$'][n]/j*ComSR-ComB)/j*100  # calculate % of profit rub
#B

In [23]:
#B.loc[28:,['profit', '%', 'ostatok']]

In [ ]:
B[['profit', '%', 'ostatok']].to_excel('./binan_prog.xlsx')